In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import gensim
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from collections import Counter
import matplotlib.pyplot as plt
from itertools import chain
import string
from pprint import pprint
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from datetime import datetime
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from dateutil import parser
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# Import Data

In [3]:
root_dir = '/content/drive/MyDrive/audace_trial/corpus/french_modern'

In [4]:
out_dir = "/content/drive/MyDrive/audace_trial/cleaned_corpus/french_modern/"

# Data Cleaning

In [22]:
def get_files(root_dir):
  # Define a list to store the text from .rtf files
  file_list = []

  # Recursively search for files in the root directory and its subdirectories
  for subdir, dirs, files in os.walk(root_dir):
      for fil in files:
          obj = {}
          file_path = os.path.join(subdir, fil)
          f = open(file_path, 'r')
          text = f.read()
          obj['text'] = text
          obj['source'] = fil.replace('.txt', '')
          print(obj['source'])
          f.close
          file_list.append(obj)
  return file_list

In [24]:
def extract_doc (file_list):
  date_pattern = r'\d{2}/\d{2}/\d{4}, \d{2}:\d{2}\n\n'
  doc_list = []
  for obj in file_list:
    entries = re.split(date_pattern, obj['text'])[1:]
    for entry in entries:
      doc = {}
      doc['text'] = entry.strip()
      doc['source'] = obj['source']
      doc_list.append(doc)
  return doc_list

In [32]:
def documentation(doc_list):
  cont_list = []

  for doc in doc_list:
    lines = doc['text'].split('\n')
    index = lines[-1].strip()
    t = {}
    t['full_text'] = '\n'.join(lines[1:-1]).strip()
    t['source'] = doc['source']
    t['index'] = index
    cont_list.append(t)
  return cont_list

In [35]:
def export_cont(out_dir, unique_list):
    for f in unique_list:
      source = f['source']
      index = f['index']
      if '/'in index:
        index = f['index'].replace('/', '_out_of_')
      cont = f['full_text']
      file_name = out_dir + index + "_" + source + ".txt"
      print(file_name)
      f = open(file_name, "w")
      f.write(cont)
      f.close

# Cleaning

In [23]:
file_list = get_files(root_dir)

LaPresse_LeDevoir_inondation_Quebec_20230710_20
LaPresse_LeDevoir_feudeforet_Quebec_20230515
LaPresse_LeDevoir_blizzard_Quebec_20230515
LaPresse_LeDevoir_vaguedechaleur_Quebec_20230515
LaPresse_LeDevoir_inondation_Quebec_20230710_17_18
LaPresse_LeDevoir_inondation_Quebec_20230710_11_13
LaPresse_LeDevoir_inondation_Quebec_20230710_19
LaPresse_LeDevoir_inondation_Quebec_20230710_00_05
LaPresse_LeDevoir_inondation_Quebec_20230710_14_16
LaPresse_LeDevoir_inondation_Quebec_20230710_06_10


In [25]:
doc_list = extract_doc(file_list)

In [33]:
cont_list = documentation(doc_list)

In [36]:
export_cont(out_dir, cont_list)

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/audace_trial/cleaned_corpus/french_modern/586_out_of_1805_LaPresse_LeDevoir_inondation_Quebec_20230710_19.txt
/content/drive/MyDrive/audace_trial/cleaned_corpus/french_modern/587_out_of_1805_LaPresse_LeDevoir_inondation_Quebec_20230710_19.txt
/content/drive/MyDrive/audace_trial/cleaned_corpus/french_modern/588_out_of_1805_LaPresse_LeDevoir_inondation_Quebec_20230710_19.txt
/content/drive/MyDrive/audace_trial/cleaned_corpus/french_modern/589_out_of_1805_LaPresse_LeDevoir_inondation_Quebec_20230710_19.txt
/content/drive/MyDrive/audace_trial/cleaned_corpus/french_modern/590_out_of_1805_LaPresse_LeDevoir_inondation_Quebec_20230710_19.txt
/content/drive/MyDrive/audace_trial/cleaned_corpus/french_modern/591_out_of_1805_LaPresse_LeDevoir_inondation_Quebec_20230710_19.txt
/content/drive/MyDrive/audace_trial/cleaned_corpus/french_modern/592_out_of_1805_LaPresse_LeDevoir_inondation_Quebec_20230710_19.txt
/content/drive/MyD

In [34]:
cont_list[0]

{'full_text': "La Presse Canadienne\nAvis, Nouvelles Générales, mercredi 1 janvier 2020 - 12:27:30 UTC -0500 1321 words\n\nAlso published in\n\nDecember 16, 2019\n\n-\n\nLa Presse Canadienne - Le fil radio\n\nVoici les principaux événements du mois d'avril 2019 au Canada\nLa Presse canadienne\n\n1 - Environnement Canada prévient, dans un rapport scientifique, que le Canada se réchauffe deux fois plus rapidement que\n\nle reste de la planète et que ce réchauffement est « irréversible ».\n\n1 - Début du procès opposant Média QMI au site satirique « Journal de Mourréal ».\n\n1 - Les premiers magasins de vente au détail de cannabis de l'Ontario ouvrent leurs portes.\n\n2  -  L'ex-ministre  fédérale  de  la  Justice,  Jody  Wilson-Raybould,  et  l'ex-présidente  du  Conseil  du Trésor,  Jane  Philpott,  sont\n\nexpulsées du caucus libéral.\n\n2 - Le député fédéral Tony Clement, exclu du caucus conservateur pour des relations « inappropriées » en ligne avec plus\n\nd'une femme, annonce qu'il